# Session de Recrutement Printemps 2017 - Team Tax Analytics
## Rendu de Guillaume Grelet - guillaume.grelet@telecom-paristech.fr

Date : février 2017


Ce rendu a pour but de fournir une réponse à l'exercice posé par PwC sur la gestion de tableur Excel.  
Il utilise les packages suivante : jupyter, pandas, PyQt, ipywidgets  
  
  
Notes :
Le code suivant est systématiquement commenté en anglais, comme l'usage le veut.  


In [26]:
# Import the necessary packages..

# For the widgets
%gui qt 
from IPython.display import display, Markdown, Latex
from PyQt5.QtWidgets import QFileDialog
from ipywidgets import *

# For the excel file processing...
import pandas as pd 

In [27]:
def filesearch(dir=None):
    """
    That function allows the user to select a file via a PyQt5 dialog window and return the file name as a str
    """
    if dir is None: dir ="~/" # Default path is ~/
    filename = QFileDialog.getOpenFileName(None)
    return filename[0]

In [28]:
datapath = filesearch("~/")

input_label_list = ["axe", "sous_axe", "indic", "label4", "label5", "label6"]

In [29]:
%%time
print("Processing file...")
data = pd.read_excel(datapath)
print("\nProcessing OK. Stats about processing:\n")

Processing file...

Processing OK. Stats about processing:

CPU times: user 7.28 s, sys: 175 ms, total: 7.45 s
Wall time: 7.53 s


In [30]:
found_labels = list(data)
print("Found labels are the following:")
for str in found_labels:
    print("    " + str)

Found labels are the following:
    axe
    sous_axe
    indic
    code_hospidiag
    libelle


In [33]:
intersection = set(found_labels).intersection(input_label_list)
intersection = list(intersection)

new_labels = []
for str in found_labels:
    if str not in intersection:
        new_labels.append(str)
        
md_str = "# Would you like to keep the following new labels ? You can also edit them before saving them. Just run the next input"
new_label_str = "[\"" + "\", \"".join(new_labels) + "\"]"

input_str = "label_list = input_label_list + " + new_label_str

display(Markdown(md_str))
get_ipython().set_next_input(input_str)

# Would you like to keep the following new labels ? You can also edit them before saving them. Just run the next input

In [48]:
a = widgets.Textarea(
    value="\n".join(new_labels) + "\n",
    placeholder='Type something',
    disabled=False
)

b = widgets.Button(
    description='Ok',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

display(a, b)